In [1]:
import pandas as pd
import numpy as np
import math, datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from matplotlib import style

style.use("ggplot")

In [2]:
data = pd.read_csv('oldmatches.csv')
display(data.head())

,Unnamed: 0,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,E0,9/8/2019,Liverpool,Norwich,4.0,1.0,H
1,E0,10/8/2019,West Ham,Man City,0.0,5.0,A
2,E0,10/8/2019,Bournemouth,Sheffield United,1.0,1.0,D
3,E0,10/8/2019,Burnley,Southampton,3.0,0.0,H
4,E0,10/8/2019,Crystal Palace,Everton,0.0,0.0,D


In [3]:
data = data.drop(['Unnamed: 0','Date'], axis=1)
data.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Liverpool,Norwich,4.0,1.0,H
1,West Ham,Man City,0.0,5.0,A
2,Bournemouth,Sheffield United,1.0,1.0,D
3,Burnley,Southampton,3.0,0.0,H
4,Crystal Palace,Everton,0.0,0.0,D


In [4]:
data['goaldiffirence'] = data.apply(lambda row: row.FTHG - row.FTAG, axis=1)
data.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,goaldiffirence
0,Liverpool,Norwich,4.0,1.0,H,3.0
1,West Ham,Man City,0.0,5.0,A,-5.0
2,Bournemouth,Sheffield United,1.0,1.0,D,0.0
3,Burnley,Southampton,3.0,0.0,H,3.0
4,Crystal Palace,Everton,0.0,0.0,D,0.0


## display all liverpool matches

In [6]:
from IPython.display import display

df1 = data[(data["HomeTeam"]=="Liverpool")  & (data["AwayTeam"]=="Everton")]
df2 = data[(data["AwayTeam"]=="Liverpool")  & (data["HomeTeam"]=="Everton")]

liveve = df1.iloc[:]
eveliv = df2.iloc[:] 
LIVEVE = liveve.append(eveliv)
LIVEVE.dropna()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,goaldiffirence
147,Liverpool,Everton,5.0,2.0,H,3.0
519,Liverpool,Everton,1.0,0.0,H,1.0
1047,Liverpool,Everton,3.0,1.0,H,2.0
1475,Liverpool,Everton,4.0,0.0,H,4.0
1859,Liverpool,Everton,2.0,1.0,H,1.0
2252,Liverpool,Everton,0.0,0.0,D,0.0
2569,Liverpool,Everton,1.0,2.0,A,-1.0
2953,Liverpool,Everton,1.0,1.0,D,0.0
3467,Liverpool,Everton,1.0,1.0,D,0.0
3885,Liverpool,Everton,3.0,2.0,H,1.0


# add winner column

In [7]:
shape = data.shape
data.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,goaldiffirence
0,Liverpool,Norwich,4.0,1.0,H,3.0
1,West Ham,Man City,0.0,5.0,A,-5.0
2,Bournemouth,Sheffield United,1.0,1.0,D,0.0
3,Burnley,Southampton,3.0,0.0,H,3.0
4,Crystal Palace,Everton,0.0,0.0,D,0.0


In [8]:
results = []
for i in data['FTR']:
    if i == 'A':
        results.append('2')
    elif i == 'H':
        results.append('1')
    else:
        results.append('0')
data['winner'] = results
data = data.drop(['FTR'], axis=1)
data.head()

,HomeTeam,AwayTeam,FTHG,FTAG,goaldiffirence,winner
0,Liverpool,Norwich,4.0,1.0,3.0,1
1,West Ham,Man City,0.0,5.0,-5.0,2
2,Bournemouth,Sheffield United,1.0,1.0,0.0,0
3,Burnley,Southampton,3.0,0.0,3.0,1
4,Crystal Palace,Everton,0.0,0.0,0.0,0


In [9]:

LIVCHE.head()


,HomeTeam,AwayTeam,FTHG,FTAG,FTR,goaldiffirence
147,Liverpool,Everton,5.0,2.0,H,3.0
519,Liverpool,Everton,1.0,0.0,H,1.0
1047,Liverpool,Everton,3.0,1.0,H,2.0
1475,Liverpool,Everton,4.0,0.0,H,4.0
1859,Liverpool,Everton,2.0,1.0,H,1.0


In [10]:

dummies = pd.get_dummies(LIVCHE.HomeTeam,LIVCHE.AwayTeam)
dummies
merged = pd.concat([dummies,LIVCHE], axis='columns')
home = merged["HomeTeam"]
away = merged["AwayTeam"]
homeaway = pd.concat([home,away], axis='columns')

final = merged.drop(['HomeTeam','AwayTeam'], axis="columns")
final.head()

,"147 Everton\n519 Everton\n1047 Everton\n1475 Everton\n1859 Everton\n2252 Everton\n2569 Everton\n2953 Everton\n3467 Everton\n3885 Everton\n4049 Everton\n4607 Everton\n4948 Everton\n5401 Everton\n5784 Everton\n6115 Everton\n6562 Everton\n6840 Everton\n7239 Everton\n7602 Everton\n8043 Everton\n8389 Everton\n298 Liverpool\n666 Liverpool\n929 Liverpool\n1218 Liverpool\n1599 Liverpool\n2144 Liverpool\n2794 Liverpool\n3157 Liverpool\n3304 Liverpool\n3665 Liverpool\n4297 Liverpool\n4382 Liverpool\n4755 Liverpool\n5264 Liverpool\n5663 Liverpool\n5895 Liverpool\n6334 Liverpool\n6675 Liverpool\n7141 Liverpool\n7462 Liverpool\n7825 Liverpool\n8186 Liverpool\nName: AwayTeam, dtype: object_Everton","147 Everton\n519 Everton\n1047 Everton\n1475 Everton\n1859 Everton\n2252 Everton\n2569 Everton\n2953 Everton\n3467 Everton\n3885 Everton\n4049 Everton\n4607 Everton\n4948 Everton\n5401 Everton\n5784 Everton\n6115 Everton\n6562 Everton\n6840 Everton\n7239 Everton\n7602 Everton\n8043 Everton\n8389 Everton\n298 Liverpool\n666 Liverpool\n929 Liverpool\n1218 Liverpool\n1599 Liverpool\n2144 Liverpool\n2794 Liverpool\n3157 Liverpool\n3304 Liverpool\n3665 Liverpool\n4297 Liverpool\n4382 Liverpool\n4755 Liverpool\n5264 Liverpool\n5663 Liverpool\n5895 Liverpool\n6334 Liverpool\n6675 Liverpool\n7141 Liverpool\n7462 Liverpool\n7825 Liverpool\n8186 Liverpool\nName: AwayTeam, dtype: object_Liverpool",FTHG,FTAG,FTR,goaldiffirence
147,0,1,5.0,2.0,H,3.0
519,0,1,1.0,0.0,H,1.0
1047,0,1,3.0,1.0,H,2.0
1475,0,1,4.0,0.0,H,4.0
1859,0,1,2.0,1.0,H,1.0


In [11]:
X = final.drop(['FTHG','FTAG'], axis=1)
X

,"147 Everton\n519 Everton\n1047 Everton\n1475 Everton\n1859 Everton\n2252 Everton\n2569 Everton\n2953 Everton\n3467 Everton\n3885 Everton\n4049 Everton\n4607 Everton\n4948 Everton\n5401 Everton\n5784 Everton\n6115 Everton\n6562 Everton\n6840 Everton\n7239 Everton\n7602 Everton\n8043 Everton\n8389 Everton\n298 Liverpool\n666 Liverpool\n929 Liverpool\n1218 Liverpool\n1599 Liverpool\n2144 Liverpool\n2794 Liverpool\n3157 Liverpool\n3304 Liverpool\n3665 Liverpool\n4297 Liverpool\n4382 Liverpool\n4755 Liverpool\n5264 Liverpool\n5663 Liverpool\n5895 Liverpool\n6334 Liverpool\n6675 Liverpool\n7141 Liverpool\n7462 Liverpool\n7825 Liverpool\n8186 Liverpool\nName: AwayTeam, dtype: object_Everton","147 Everton\n519 Everton\n1047 Everton\n1475 Everton\n1859 Everton\n2252 Everton\n2569 Everton\n2953 Everton\n3467 Everton\n3885 Everton\n4049 Everton\n4607 Everton\n4948 Everton\n5401 Everton\n5784 Everton\n6115 Everton\n6562 Everton\n6840 Everton\n7239 Everton\n7602 Everton\n8043 Everton\n8389 Everton\n298 Liverpool\n666 Liverpool\n929 Liverpool\n1218 Liverpool\n1599 Liverpool\n2144 Liverpool\n2794 Liverpool\n3157 Liverpool\n3304 Liverpool\n3665 Liverpool\n4297 Liverpool\n4382 Liverpool\n4755 Liverpool\n5264 Liverpool\n5663 Liverpool\n5895 Liverpool\n6334 Liverpool\n6675 Liverpool\n7141 Liverpool\n7462 Liverpool\n7825 Liverpool\n8186 Liverpool\nName: AwayTeam, dtype: object_Liverpool",FTR,goaldiffirence
147,0,1,H,3.0
519,0,1,H,1.0
1047,0,1,H,2.0
1475,0,1,H,4.0
1859,0,1,H,1.0
2252,0,1,D,0.0
2569,0,1,A,-1.0
2953,0,1,D,0.0
3467,0,1,D,0.0
3885,0,1,H,1.0


### PREDICTING COLUMNS

In [12]:
y1 = final['FTHG']
y2 = final['FTAG']
y = pd.concat([y1,y2], axis="columns")


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,shuffle=False)
#Xtrain, Xtest, ytrain, ytest = train_test_split(home,away, test_size=0.03,shuffle=False)

newhomeaway = pd.concat([X_test,y_test], axis="columns") 
newhomeaway

,"147 Everton\n519 Everton\n1047 Everton\n1475 Everton\n1859 Everton\n2252 Everton\n2569 Everton\n2953 Everton\n3467 Everton\n3885 Everton\n4049 Everton\n4607 Everton\n4948 Everton\n5401 Everton\n5784 Everton\n6115 Everton\n6562 Everton\n6840 Everton\n7239 Everton\n7602 Everton\n8043 Everton\n8389 Everton\n298 Liverpool\n666 Liverpool\n929 Liverpool\n1218 Liverpool\n1599 Liverpool\n2144 Liverpool\n2794 Liverpool\n3157 Liverpool\n3304 Liverpool\n3665 Liverpool\n4297 Liverpool\n4382 Liverpool\n4755 Liverpool\n5264 Liverpool\n5663 Liverpool\n5895 Liverpool\n6334 Liverpool\n6675 Liverpool\n7141 Liverpool\n7462 Liverpool\n7825 Liverpool\n8186 Liverpool\nName: AwayTeam, dtype: object_Everton","147 Everton\n519 Everton\n1047 Everton\n1475 Everton\n1859 Everton\n2252 Everton\n2569 Everton\n2953 Everton\n3467 Everton\n3885 Everton\n4049 Everton\n4607 Everton\n4948 Everton\n5401 Everton\n5784 Everton\n6115 Everton\n6562 Everton\n6840 Everton\n7239 Everton\n7602 Everton\n8043 Everton\n8389 Everton\n298 Liverpool\n666 Liverpool\n929 Liverpool\n1218 Liverpool\n1599 Liverpool\n2144 Liverpool\n2794 Liverpool\n3157 Liverpool\n3304 Liverpool\n3665 Liverpool\n4297 Liverpool\n4382 Liverpool\n4755 Liverpool\n5264 Liverpool\n5663 Liverpool\n5895 Liverpool\n6334 Liverpool\n6675 Liverpool\n7141 Liverpool\n7462 Liverpool\n7825 Liverpool\n8186 Liverpool\nName: AwayTeam, dtype: object_Liverpool",FTR,goaldiffirence,FTHG,FTAG
3304,1,0,H,2.0,2.0,0.0
3665,1,0,D,0.0,0.0,0.0
4297,1,0,D,0.0,0.0,0.0
4382,1,0,A,-2.0,1.0,3.0
4755,1,0,A,-3.0,0.0,3.0
5264,1,0,H,1.0,1.0,0.0
5663,1,0,A,-2.0,1.0,3.0
5895,1,0,H,3.0,3.0,0.0
6334,1,0,A,-1.0,1.0,2.0
6675,1,0,A,-2.0,0.0,2.0


In [14]:
data.isnull().sum().sum()
data.dropna()


,HomeTeam,AwayTeam,FTHG,FTAG,goaldiffirence,winner
0,Liverpool,Norwich,4.0,1.0,3.0,1
1,West Ham,Man City,0.0,5.0,-5.0,2
2,Bournemouth,Sheffield United,1.0,1.0,0.0,0
3,Burnley,Southampton,3.0,0.0,3.0,1
4,Crystal Palace,Everton,0.0,0.0,0.0,0
...,...,...,...,...,...,...
8139,Sunderland,Man United,0.0,1.0,-1.0,2
8140,Swansea,Liverpool,1.0,0.0,1.0,1
8141,Tottenham,Fulham,2.0,0.0,2.0,1
8142,West Brom,Arsenal,2.0,3.0,-1.0,2


In [15]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
ypred = lr.predict(X_test)
# convert array to dataframe
ypred = pd.DataFrame(ypred)
newhomeaway["FTHG"] = y_test["FTHG"]
newhomeaway["FTAG"] = y_test["FTAG"]
newhomeaway["predFTHG"] = ypred[0].values
newhomeaway["predFTAG"] = ypred[1].values

newhomeaway

ValueError: could not convert string to float: 'H'

In [16]:
a = newhomeaway["predFTHG"].sum()
shapes = newhomeaway["HomeTeam"].shape
print("hometeam mean   :",float(a/shapes))
b = newhomeaway["predFTAG"].sum()
print("awayteam mean   : ",float(b/shapes))

KeyError: 'predFTHG'

## reading csv files

In [243]:
ranking = pd.read_csv('ranking.csv')
ranking.head()

,Position,team,Point
0,1,Liverpool,2010
1,2,Man City,1900
2,3,Chelsea,1800
3,4,Man United,1798
4,5,Arsenal,1788


In [244]:
ranking = pd.read_csv('ranking.csv')
fixtures = pd.read_csv('fixtures.csv')
random = pd.read_csv('random.csv')

random.head()

,Home Team,Away Team,Result
0,Fulham,Arsenal,Arsenal
1,Burnley,Man Utd,Man Utd
2,Man City,Aston Villa,Aston Villa
3,Crystal Palace,Southampton,Southampton
4,Liverpool,Leeds,Leeds


In [245]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)
rf.fit(X_train, y_train)


score = rf.score(X_train, y_train)
score2 = rf.score( X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Testing set accuracy: ", '%.3f'%(score2))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [246]:
fixtures.insert(1, 'firstposition', fixtures['Home Team'].map(ranking.set_index('team')['Position']))
fixtures.insert(2, 'secondposition', fixtures['Away Team'].map(ranking.set_index('team')['Position']))
fixtures.head()


,Round Number,firstposition,secondposition,Date,Location,Home Team,Away Team,Result
0,1,20.0,5.0,12/9/2020 12:30,Craven Cottage,Fulham,Arsenal,NaN
1,1,21.0,NaN,12/9/2020 15:00,Turf Moor,Burnley,Man Utd,NaN
2,1,2.0,19.0,12/9/2020 15:00,Etihad Stadium,Man City,Aston Villa,NaN
3,1,15.0,18.0,12/9/2020 15:00,Selhurst Park,Crystal Palace,Southampton,NaN
4,1,1.0,11.0,12/9/2020 17:30,Anfield,Liverpool,Leeds,NaN


In [247]:
predset = []
for index,row in fixtures.iterrows():
    if row['firstposition'] < row['secondposition']:
        predset.append({'Home Team': row['Home Team'], 'Away Team': row['Away Team'], 'winningteam':None})
    else:
        predset.append({'Home Team': row['Away Team'], 'Away Team': row['Home Team'], 'winningteam':None})
predset = pd.DataFrame(predset)
backuppredset = predset
predset.head()

,Home Team,Away Team,winningteam
0,Arsenal,Fulham,None
1,Man Utd,Burnley,None
2,Man City,Aston Villa,None
3,Crystal Palace,Southampton,None
4,Liverpool,Leeds,None


## get dummy variables and drop winning team column

In [248]:
predset = pd.get_dummies(predset, prefix=['Home Team', 'Away Team'], columns=['Home Team','Away Team'])
# add missing columns compared to the model's traning data set
missingcolumns = set(FINAL.columns) - set(predset.columns)
for c in missingcolumns:
    predset[c] = 0
predset = predset[FINAL.columns]

predset = predset.drop(['winner'], axis=1)
predset.head()

NameError: name 'FINAL' is not defined

##### Liverpool,Man City,Chelsea,Man Utd,Arsenal,Wolwes,Spurs,Leicester,Everton,Brighton,Leeds,Sheffield Utd,
##### Newcastle,West Brom,Crystal Palace,West Ham,Cardiff,Southampton,Aston Villa,Fulham,Burnley,Stoke City,Norwich,Watford,
##### Wimbledon,Bournemouth,Blackburn,Sunderland,QPR,Wigan,Swansea,Swindon,Bolton,Blackpool,
##### Birmingham,Portsmouth,Hull,Derby,Middlesbrough,Reading,Charlton,Ipswich,Nott'm Forest,Coventry,Oldham,Barnsley,Bradford

In [145]:
print('fixtures: ',predset.shape)
print(fixtures.shape[0])

fixtures:  (380, 98)
380


In [154]:
count = 0

tablelist = ['Liverpool','Man City','Chelsea','Man Utd','Arsenal','Wolwes',
        'Spurs','Leicester','Everton','Brighton','Leeds','Sheffield Utd',
         'Newcastle','West Brom','Crystal Palace','West Ham','Cardiff',
        'Southampton','Aston Villa','Fulham','Burnley','Stoke City',
         'Norwich','Watford','Wimbledon','Bournemouth','Blackburn',
         'Sunderland','QPR','Wigan','Swansea','Swindon','Bolton',
          'Blackpool','Birmingham','Portsmouth','Hull','Derby',
        'Middlesbrough','Reading','Charlton','Ipswich',
        'Nottm Forest','Coventry','Oldham','Barnsley','Bradford']
for i in range(0,47):
    tablelist.append(0)

tablelist[i]

    

predictions = rf.predict(predset)
 
for i in range(fixtures.shape[0]):
    print(backuppredset.iloc[i,1], ' AND ', backuppredset.iloc[i,0])
    if predictions[i] == 1:
        print('winner: ', backuppredset.iloc[i,1])
        count = count+1
    else:
        print('winner: ', backuppredset.iloc[i,0])
        count = count+1
    print(" ")
print("mac sayisi:" ,count)

Liverpool
Man City
Chelsea
Man Utd
Arsenal
Wolwes
Spurs
Leicester
Everton
Brighton
Leeds
Sheffield Utd
Newcastle
West Brom
Crystal Palace
West Ham
Cardiff
Southampton
Aston Villa
Fulham
Burnley
Stoke City
Norwich
Watford
Wimbledon
Bournemouth
Blackburn
Sunderland
QPR
Wigan
Swansea
Swindon
Bolton
Blackpool
Birmingham
Portsmouth
Hull
Derby
Middlesbrough
Reading
Charlton
Ipswich
Nottm Forest
Coventry
Oldham
Barnsley
Bradford
Fulham  AND  Arsenal
winner:  Arsenal
 
Burnley  AND  Man Utd
winner:  Man Utd
 
Aston Villa  AND  Man City
winner:  Man City
 
Southampton  AND  Crystal Palace
winner:  Crystal Palace
 
Leeds  AND  Liverpool
winner:  Liverpool
 
West Ham  AND  Newcastle
winner:  Newcastle
 
West Brom  AND  Leicester
winner:  Leicester
 
Everton  AND  Spurs
winner:  Spurs
 
Sheffield Utd  AND  Wolves
winner:  Wolves
 
Brighton  AND  Chelsea
winner:  Chelsea
 
West Brom  AND  Everton
winner:  Everton
 
Fulham  AND  Leeds
winner:  Leeds
 
Crystal Palace  AND  Man Utd
winner:  Man Utd
 


Leeds  AND  Chelsea
winner:  Chelsea
 
Sheffield Utd  AND  Leicester
winner:  Leicester
 
West Ham  AND  Man Utd
winner:  Man Utd
 
Aston Villa  AND  Newcastle
winner:  Newcastle
 
Southampton  AND  Brighton
winner:  Brighton
 
Wolves  AND  Liverpool
winner:  Liverpool
 
Newcastle  AND  Brighton
winner:  Brighton
 
Burnley  AND  Leicester
winner:  Leicester
 
Crystal Palace  AND  Man Utd
winner:  Man Utd
 
Fulham  AND  Leeds
winner:  Leeds
 
Chelsea  AND  Liverpool
winner:  Liverpool
 
Man City  AND  Wolves
winner:  Wolves
 
Aston Villa  AND  Sheffield Utd
winner:  Sheffield Utd
 
Southampton  AND  Spurs
winner:  Spurs
 
West Brom  AND  Everton
winner:  Everton
 
West Ham  AND  Arsenal
winner:  Arsenal
 
Arsenal  AND  Liverpool
winner:  Liverpool
 
Fulham  AND  Aston Villa
winner:  Aston Villa
 
West Brom  AND  Chelsea
winner:  Chelsea
 
Crystal Palace  AND  Everton
winner:  Everton
 
Sheffield Utd  AND  Leeds
winner:  Leeds
 
Leicester  AND  Man City
winner:  Man City
 
Brighton  AND 